In [1]:
import os
import cx_Oracle as db
import requests
import json
import datetime
import time
import pandas as pd

In [2]:
LOCATION = 'C:' + os.environ['HOMEPATH'] + '\\instantclient_21_6'
os.environ['PATH'] = LOCATION + ';' + os.environ['PATH']
os.environ['NLS_LANG'] = 'KOREAN_KOREA.KO16MSWIN949'

conn = db.connect('lolpredict', 'korhrd', '163.152.224.168:1521')
cursor = conn.cursor()

In [3]:
tablesize = 20000
i = 6

startRownum = tablesize * i
endRownum = tablesize * (i + 1)

sql = "select * from matchinfo OFFSET {} ROWS FETCH NEXT {} ROWS ONLY".format(startRownum, endRownum)
cursor.execute(sql)
matchList = cursor.fetchall()

matchConvList = []
for match in matchList:
    resultStr = ""
    for i in match[1:]:
        if type(i) == type(resultStr):
            resultStr += i

    try:
        ex = eval(resultStr.replace("\\\"", "'"))
    except:
        continue

    startTime = ex['info']['gameStartTimestamp'] / 1000
    realTime = datetime.datetime.fromtimestamp(startTime).strftime("%y%m%d")

    if ((ex['info']['queueId'] == 420)
        & (realTime > "220524")):
        for i in ex['info']['participants']:
            matchConv = {"matchID" : match[0],
                        "championID" :  i['championId'], 
                        "position" : i['teamPosition'],
                        "win" : i['win'],
                        "kda" : i['challenges']['kda']}
            matchConvList.append(matchConv)

matchdf = pd.DataFrame(matchConvList)
matchdf.to_csv('data/champions{}.csv'.format(startRownum))